### Introduction/Business Problem:

Mr X and Mrs Y are taking their kids on a family trip to New York City for one week. For this special weekend everyone wants to do something special they will remember forever. The kids want to go to an exhibit Zoo, Mrs Y wants to be close to a Chinese restaurant and Mr X desires is to  be next to gym so he can keep working out even during the family trip. So the family wants a hotel that matches will all their needs. As a data scientist they called me so I can help them to reach their expectations.


In [2]:
import numpy as np #library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries successfully imported.')

Libraries successfully imported.


### Geolocation of NYC

In [3]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Foursquare
#### Retrieving data for hotels in an area of 20 km

In [98]:
client_id = 'ACFKMFU1NTMBCQ30D2GZWVL3JHIOBDNESJNFLINFSES0PJRM' # your Foursquare ID
client_secret = '11541GPGNXMHAUYG4HIB020A2W1RDU52YKTLUHWY1A2XNTOF' # your Foursquare Secret
version = '20180604'
radius = 20000
query= "hotel"
limit=200
categoryId ="4bf58dd8d48988d1fa931735"
url_root = " https://api.foursquare.com/v2/venues/search?"

In [99]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(client_id, client_secret, latitude, longitude, version, query, radius, limit,categoryId)
url

'https://api.foursquare.com/v2/venues/search?client_id=ACFKMFU1NTMBCQ30D2GZWVL3JHIOBDNESJNFLINFSES0PJRM&client_secret=11541GPGNXMHAUYG4HIB020A2W1RDU52YKTLUHWY1A2XNTOF&ll=40.71513343509779,-73.98117571779109&v=20180604&query=hotel&radius=20000&limit=200&categoryId=4bf58dd8d48988d1fa931735'

In [100]:
import requests
results = requests.get(url).json()
results['response']['venues']

[{'id': '54c0151d498e4c827296cd41',
  'name': 'Hotel Indigo',
  'location': {'address': '171 Ludlow St',
   'crossStreet': 'btw Houston & Stanton',
   'lat': 40.721762215432754,
   'lng': -73.9880921645,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.721762215432754,
     'lng': -73.9880921645},
    {'label': 'entrance', 'lat': 40.721735, 'lng': -73.987701}],
   'distance': 940,
   'postalCode': '10002',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['171 Ludlow St (btw Houston & Stanton)',
    'New York, NY 10002',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1fa931735',
    'name': 'Hotel',
    'pluralName': 'Hotels',
    'shortName': 'Hotel',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1595126817',
  'hasPerk': False},
 {'id': '578692f4498e1054905dbde7',
  'name': 'Hotel 50 Bowery NYC',
  'lo

In [101]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
venues = results['response']['venues']

# tranform venues into a dataframe
df1 = json_normalize(venues)
df1.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,54c0151d498e4c827296cd41,Hotel Indigo,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1595126817,False,171 Ludlow St,btw Houston & Stanton,40.721762,-73.988092,"[{'label': 'display', 'lat': 40.72176221543275...",940,10002,US,New York,NY,United States,"[171 Ludlow St (btw Houston & Stanton), New Yo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,578692f4498e1054905dbde7,Hotel 50 Bowery NYC,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1595126817,False,50 Bowery,btwn Bayard & Canal St,40.715936,-73.996789,"[{'label': 'display', 'lat': 40.7159364, 'lng'...",1320,10013,US,New York,NY,United States,"[50 Bowery (btwn Bayard & Canal St), New York,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,43616a00f964a52030291fe3,Hotel on Rivington,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1595126817,False,107 Rivington St,btwn Ludlow & Essex St,40.719953,-73.988107,"[{'label': 'display', 'lat': 40.71995259064954...",793,10002,US,New York,NY,United States,"[107 Rivington St (btwn Ludlow & Essex St), Ne...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,536020eb11d2ce653fb711d0,The Ludlow Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1595126817,False,180 Ludlow St,NaN,40.721857,-73.987204,"[{'label': 'display', 'lat': 40.7218567, 'lng'...",904,10002,US,New York,NY,United States,"[180 Ludlow St, New York, NY 10002, United Sta...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4b5668d6f964a520370f28e3,SIXTY LES Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1595126817,False,190 Allen St,btwn Stanton & E Houston St,40.722141,-73.988571,"[{'label': 'display', 'lat': 40.72214080000001...",998,10002,US,New York,NY,United States,"[190 Allen St (btwn Stanton & E Houston St), N...",78329156,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
df2 = df1[["name","location.address","location.lat","location.lng","location.formattedAddress"]]

In [103]:
df2.rename(columns={"location.address": "address","location.lat":"Latitude","location.lng":"Longitude","location.formattedAddress":"full address"}, errors="raise", inplace=True)

### Final data frame for hotels

In [104]:
df2.head()

,name,address,Latitude,Longitude,full address
0,Hotel Indigo,171 Ludlow St,40.721762,-73.988092,"[171 Ludlow St (btw Houston & Stanton), New Yo..."
1,Hotel 50 Bowery NYC,50 Bowery,40.715936,-73.996789,"[50 Bowery (btwn Bayard & Canal St), New York,..."
2,Hotel on Rivington,107 Rivington St,40.719953,-73.988107,"[107 Rivington St (btwn Ludlow & Essex St), Ne..."
3,The Ludlow Hotel,180 Ludlow St,40.721857,-73.987204,"[180 Ludlow St, New York, NY 10002, United Sta..."
4,SIXTY LES Hotel,190 Allen St,40.722141,-73.988571,"[190 Allen St (btwn Stanton & E Houston St), N..."


In [105]:
df2.shape

(50, 5)

In [106]:
df2.dropna(inplace=True)

C:\Users\jake brown\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [107]:
df2.shape

(50, 5)

### Map of NYC with the hotels we choose

In [108]:
ny_map = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for name, latitude, longitude in zip(df2['name'], df2['Latitude'], df2['Longitude']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='gray',
        fill_opacity=0.7,
        parse_html=False).add_to(ny_map)  
      
ny_map

### Foursquare
#### Retrieving data for zoos in an area of 20 km

In [26]:
client_id = 'ACFKMFU1NTMBCQ30D2GZWVL3JHIOBDNESJNFLINFSES0PJRM' # your Foursquare ID
client_secret = '11541GPGNXMHAUYG4HIB020A2W1RDU52YKTLUHWY1A2XNTOF' # your Foursquare Secret
version = '20180604'
radius = 20000
query= "zoo"
limit=200
categoryId ="4bf58dd8d48988d17b941735"
url_root = " https://api.foursquare.com/v2/venues/search?"

In [27]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(client_id, client_secret, latitude, longitude, version, query, radius, limit,categoryId)
url2

'https://api.foursquare.com/v2/venues/search?client_id=ACFKMFU1NTMBCQ30D2GZWVL3JHIOBDNESJNFLINFSES0PJRM&client_secret=11541GPGNXMHAUYG4HIB020A2W1RDU52YKTLUHWY1A2XNTOF&ll=40.6655972949291,-73.96476364174872&v=20180604&query=zoo&radius=20000&limit=200&categoryId=4bf58dd8d48988d17b941735'

In [28]:
import requests
results2 = requests.get(url2).json()
results2['response']['venues']

[{'id': '49d8307ef964a520b15d1fe3',
  'name': 'Prospect Park Zoo',
  'location': {'address': '450 Flatbush Ave',
   'crossStreet': 'at Ocean Ave.',
   'lat': 40.665014570215,
   'lng': -73.96446704864502,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.665014570215,
     'lng': -73.96446704864502},
    {'label': 'entrance', 'lat': 40.666031, 'lng': -73.964958}],
   'distance': 69,
   'postalCode': '11225',
   'cc': 'US',
   'city': 'Brooklyn',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['450 Flatbush Ave (at Ocean Ave.)',
    'Brooklyn, NY 11225',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d17b941735',
    'name': 'Zoo',
    'pluralName': 'Zoos',
    'shortName': 'Zoo',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/zoo_',
     'suffix': '.png'},
    'primary': True}],
  'venuePage': {'id': '78154898'},
  'referralId': 'v-1595124332',
  'hasPerk': False},
 {'id': '4dab47b143a11281970c75e7',
  '

In [29]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
venues2 = results2['response']['venues']

# tranform venues into a dataframe
df3 = json_normalize(venues2)
df3

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,49d8307ef964a520b15d1fe3,Prospect Park Zoo,"[{'id': '4bf58dd8d48988d17b941735', 'name': 'Z...",v-1595124332,False,450 Flatbush Ave,at Ocean Ave.,40.665015,-73.964467,"[{'label': 'display', 'lat': 40.665014570215, ...",69,11225,US,Brooklyn,NY,United States,"[450 Flatbush Ave (at Ocean Ave.), Brooklyn, N...",78154898,NaN
1,4dab47b143a11281970c75e7,Sea Lion Court @ Prospect Park Zoo,"[{'id': '58daa1558bbb0b01f18ec1fd', 'name': 'Z...",v-1595124332,False,Flatbush Ave,NaN,40.665597,-73.964764,"[{'label': 'display', 'lat': 40.6655972949291,...",0,11238,US,Brooklyn,NY,United States,"[Flatbush Ave, Brooklyn, NY 11238, United States]",NaN,NaN
2,4a09f665f964a52077741fe3,Central Park Zoo,"[{'id': '4bf58dd8d48988d17b941735', 'name': 'Z...",v-1595124332,False,Central Park,btwn E 63rd & E 65th St,40.767655,-73.971827,"[{'label': 'display', 'lat': 40.76765542900379...",11376,10009,US,New York,NY,United States,"[Central Park (btwn E 63rd & E 65th St), New Y...",124456052,NaN
3,4492ad65f964a52075341fe3,Bronx Zoo,"[{'id': '4bf58dd8d48988d17b941735', 'name': 'Z...",v-1595124332,False,2300 Southern Blvd,at Boston Rd,40.853107,-73.878094,"[{'label': 'routing', 'lat': 40.84401833985193...",22115,10460,US,Bronx,NY,United States,"[2300 Southern Blvd (at Boston Rd), Bronx, NY ...",35584906,NaN
4,4bc22d23461576b081fd7c32,Central Park Zoo - Tisch Children's Zoo,"[{'id': '4bf58dd8d48988d17b941735', 'name': 'Z...",v-1595124332,False,66th Street & 5th Avenue,NaN,40.768509,-73.970561,"[{'label': 'display', 'lat': 40.76850861489573...",11466,10065,US,New York,NY,United States,"[66th Street & 5th Avenue, New York, NY 10065,...",NaN,NaN
5,49f21ff8f964a520eb691fe3,Staten Island Zoo,"[{'id': '4bf58dd8d48988d17b941735', 'name': 'Z...",v-1595124332,False,614 Broadway,at Colonial Ct,40.625083,-74.115752,"[{'label': 'display', 'lat': 40.62508310821342...",13527,10310,US,Staten Island,NY,United States,"[614 Broadway (at Colonial Ct), Staten Island,...",NaN,NaN
6,4dbadb926e815ab0de5bf9a6,Central Park Zoo - Penguins & Seabirds,"[{'id': '58daa1558bbb0b01f18ec1fd', 'name': 'Z...",v-1595124332,False,64th St,5th Ave,40.767745,-73.971826,"[{'label': 'display', 'lat': 40.76774458176332...",11386,10021,US,New York,NY,United States,"[64th St (5th Ave), New York, NY 10021, United...",NaN,Central Park
7,5bb124e43149b9002c7da00c,Central Park Zoo - Grizzly Bears,"[{'id': '58daa1558bbb0b01f18ec1fd', 'name': 'Z...",v-1595124332,False,NaN,NaN,40.767827,-73.971921,"[{'label': 'display', 'lat': 40.767827, 'lng':...",11396,10028,US,New York,NY,United States,"[New York, NY 10028, United States]",NaN,Central Park
8,4dfb986e18386e743d98c038,Central Park Zoo - Snow Monkey,"[{'id': '58daa1558bbb0b01f18ec1fd', 'name': 'Z...",v-1595124332,False,NaN,NaN,40.767914,-73.972204,"[{'label': 'display', 'lat': 40.76791361297603...",11407,NaN,US,New York,NY,United States,"[New York, NY, United States]",NaN,NaN
9,4f9d85f2e4b087d95e3bd792,Central Park Zoo - Red Panda,"[{'id': '58daa1558bbb0b01f18ec1fd', 'name': 'Z...",v-1595124332,False,NaN,NaN,40.768099,-73.972626,"[{'label': 'display', 'lat': 40.76809875771227...",11429,10021,US,New York,NY,United States,"[New York, NY 10021, United States]",NaN,NaN


In [30]:
df3.shape

(27, 19)

In [31]:
df3_= df3[["name","location.address","location.lat","location.lng","location.formattedAddress"]]

In [32]:
df3_.rename(columns={"location.address": "address","location.lat":"Latitude","location.lng":"Longitude","location.formattedAddress":"full address"}, errors="raise", inplace=True)

### Final data frame for ZOOs

In [33]:
df3_.head()

,name,address,Latitude,Longitude,full address
0,Prospect Park Zoo,450 Flatbush Ave,40.665015,-73.964467,"[450 Flatbush Ave (at Ocean Ave.), Brooklyn, N..."
1,Sea Lion Court @ Prospect Park Zoo,Flatbush Ave,40.665597,-73.964764,"[Flatbush Ave, Brooklyn, NY 11238, United States]"
2,Central Park Zoo,Central Park,40.767655,-73.971827,"[Central Park (btwn E 63rd & E 65th St), New Y..."
3,Bronx Zoo,2300 Southern Blvd,40.853107,-73.878094,"[2300 Southern Blvd (at Boston Rd), Bronx, NY ..."
4,Central Park Zoo - Tisch Children's Zoo,66th Street & 5th Avenue,40.768509,-73.970561,"[66th Street & 5th Avenue, New York, NY 10065,..."


In [34]:
df3_.dropna(inplace=True)

C:\Users\jake brown\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
df3_.shape

(20, 5)

#### Map of NYC with the zoos with picked

In [92]:
ny_map2 = folium.Map(location=[latitude, longitude], zoom_start=10.7)

# add markers to map
for name, latitude, longitude in zip(df3_['name'], df3_['Latitude'], df3_['Longitude']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color='gray',
        fill_opacity=0.7,
        parse_html=False).add_to(ny_map2)  
      
ny_map2

### Foursquare
#### Retrieving data for Chinese restaurants in an area of 20 km

In [46]:
client_id = 'ACFKMFU1NTMBCQ30D2GZWVL3JHIOBDNESJNFLINFSES0PJRM' # your Foursquare ID
client_secret = '11541GPGNXMHAUYG4HIB020A2W1RDU52YKTLUHWY1A2XNTOF' # your Foursquare Secret
version = '20180604'
radius = 2000
query= "chinese restaurant"
limit=200
categoryId ="4bf58dd8d48988d145941735"
url_root = " https://api.foursquare.com/v2/venues/search?"

In [47]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(client_id, client_secret, latitude, longitude, version, query, radius, limit,categoryId)
url2

'https://api.foursquare.com/v2/venues/search?client_id=ACFKMFU1NTMBCQ30D2GZWVL3JHIOBDNESJNFLINFSES0PJRM&client_secret=11541GPGNXMHAUYG4HIB020A2W1RDU52YKTLUHWY1A2XNTOF&ll=40.852882130832036,-73.8785453754281&v=20180604&query=chinese restaurant&radius=2000&limit=200&categoryId=4bf58dd8d48988d145941735'

In [48]:
import requests
results2 = requests.get(url2).json()
results2['response']['venues']

[{'id': '4ca29134554b236a517f1f48',
  'name': 'No. 8 Chinese Restaurant',
  'location': {'address': '694 Thwaites Pl',
   'lat': 40.858553,
   'lng': -73.868267,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.858553,
     'lng': -73.868267},
    {'label': 'entrance', 'lat': 40.858608, 'lng': -73.868312}],
   'distance': 1071,
   'postalCode': '10467',
   'cc': 'US',
   'city': 'Bronx',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['694 Thwaites Pl',
    'Bronx, NY 10467',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d145941735',
    'name': 'Chinese Restaurant',
    'pluralName': 'Chinese Restaurants',
    'shortName': 'Chinese',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1595125079',
  'hasPerk': False},
 {'id': '4b9d6b45f964a52078ab36e3',
  'name': "Mr. Q's Chinese Restaurant",
  'location': {'address': '2115 Williamsbridge Rd',
  

In [49]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
venues2 = results2['response']['venues']

# tranform venues into a dataframe
df4 = json_normalize(venues2)
df4

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,4ca29134554b236a517f1f48,No. 8 Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1595125079,False,694 Thwaites Pl,40.858553,-73.868267,"[{'label': 'display', 'lat': 40.858553, 'lng':...",1071,10467,US,Bronx,NY,United States,"[694 Thwaites Pl, Bronx, NY 10467, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4b9d6b45f964a52078ab36e3,Mr. Q's Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1595125079,False,2115 Williamsbridge Rd,40.855790,-73.855455,"[{'label': 'display', 'lat': 40.85579022389548...",1970,10461,US,Bronx,NY,United States,"[2115 Williamsbridge Rd, Bronx, NY 10461, Unit...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4f4348da19834bc91f55b4f0,Lucky Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1595125079,False,653 E 182nd St,40.851396,-73.888245,"[{'label': 'display', 'lat': 40.851396, 'lng':...",833,10457,US,Bronx,NY,United States,"[653 E 182nd St, Bronx, NY 10457, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4ca0f9de03133704368f7fd5,Rainbow Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1595125079,False,613 E Tremont Ave,40.846241,-73.892555,"[{'label': 'entrance', 'lat': 40.846277, 'lng'...",1392,10457,US,Bronx,NY,United States,"[613 E Tremont Ave, Bronx, NY 10457, United St...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4e4ce1a6bd413c4cc66cebc8,Nice Food Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1595125079,False,571 E 187th St,40.856001,-73.888063,"[{'label': 'display', 'lat': 40.856001, 'lng':...",873,10458,US,Bronx,NY,United States,"[571 E 187th St, Bronx, NY 10458, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4ed4326f2c5b695abedee221,Lung Fong Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1595125079,False,"870 E 180th St, Bronx, NY 10460",40.845089,-73.883153,"[{'label': 'display', 'lat': 40.845089, 'lng':...",950,10460,US,Bronx,NY,United States,"[870 E 180th St, Bronx, NY 10460, Bronx, NY 10...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4da9c2358154abafc29ba5d8,L & Y Wing Hing Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1595125079,False,1661 Metropolitan Ave,40.840735,-73.855535,"[{'label': 'display', 'lat': 40.84073499999999...",2362,10462,US,Bronx,NY,United States,"[1661 Metropolitan Ave, Bronx, NY 10462, Unite...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4e4cf90cbd413c4cc66da8fe,Boe Lee Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1595125079,False,741 Lydig Ave,40.854793,-73.866017,"[{'label': 'display', 'lat': 40.854793, 'lng':...",1076,10462,US,Bronx,NY,United States,"[741 Lydig Ave, Bronx, NY 10462, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4e4d037fbd413c4cc66dfd25,Hing Wang Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1595125079,False,316 E 194th St,40.864382,-73.891722,"[{'label': 'display', 'lat': 40.864382, 'lng':...",1693,10458,US,Bronx,NY,United States,"[316 E 194th St, Bronx, NY 10458, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4ccf58277f56a1434cfec6a6,Wings Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1595125079,False,879 E Tremont Ave,40.843266,-73.885905,"[{'label': 'display', 'lat': 40.843266, 'lng':...",1236,10460,US,Bronx,NY,United States,"[879 E Tremont Ave, Bronx, NY 10460, United St...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df4_= df4[["name","location.address","location.lat","location.lng","location.formattedAddress"]]

In [51]:
df4_.rename(columns={"location.address": "address","location.lat":"Latitude","location.lng":"Longitude","location.formattedAddress":"full address"}, errors="raise", inplace=True)

### Final data frame of NYC with the restaurant we got

In [52]:
df4_.head()

,name,address,Latitude,Longitude,full address
0,No. 8 Chinese Restaurant,694 Thwaites Pl,40.858553,-73.868267,"[694 Thwaites Pl, Bronx, NY 10467, United States]"
1,Mr. Q's Chinese Restaurant,2115 Williamsbridge Rd,40.855790,-73.855455,"[2115 Williamsbridge Rd, Bronx, NY 10461, Unit..."
2,Lucky Chinese Restaurant,653 E 182nd St,40.851396,-73.888245,"[653 E 182nd St, Bronx, NY 10457, United States]"
3,Rainbow Chinese Restaurant,613 E Tremont Ave,40.846241,-73.892555,"[613 E Tremont Ave, Bronx, NY 10457, United St..."
4,Nice Food Chinese Restaurant,571 E 187th St,40.856001,-73.888063,"[571 E 187th St, Bronx, NY 10458, United States]"


In [53]:
df4_.shape

(50, 5)

In [54]:
df4_.dropna(inplace=True)

C:\Users\jake brown\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [55]:
df4_.shape

(48, 5)

### Map of NYC with the Restaurant we retrieved from Foursquare

In [87]:
ny_map3 = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for name, latitude, longitude in zip(df4_['name'], df4_['Latitude'], df4_['Longitude']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=4,
        popup=label,
        color='green',
        fill=True,
        fill_color='gray',
        fill_opacity=0.7,
        parse_html=False).add_to(ny_map3)  
      
ny_map3

### Foursquare
#### Retrieving data for gyms in an area of 20 km

In [72]:
client_id = 'ACFKMFU1NTMBCQ30D2GZWVL3JHIOBDNESJNFLINFSES0PJRM' # your Foursquare ID
client_secret = '11541GPGNXMHAUYG4HIB020A2W1RDU52YKTLUHWY1A2XNTOF' # your Foursquare Secret
version = '20180604'
radius = 20000
query= "gym"
limit=200
categoryId ="4bf58dd8d48988d176941735"
url_root = " https://api.foursquare.com/v2/venues/search?"

In [73]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(client_id, client_secret, latitude, longitude, version, query, radius, limit,categoryId)
url2

'https://api.foursquare.com/v2/venues/search?client_id=ACFKMFU1NTMBCQ30D2GZWVL3JHIOBDNESJNFLINFSES0PJRM&client_secret=11541GPGNXMHAUYG4HIB020A2W1RDU52YKTLUHWY1A2XNTOF&ll=40.853918,-73.899629&v=20180604&query=gym&radius=20000&limit=200&categoryId=4bf58dd8d48988d176941735'

In [74]:
import requests
results2 = requests.get(url2).json()
results2['response']['venues']

[{'id': '4cf19de26195721e9bc442c1',
  'name': "Daro's Extreme Gym",
  'location': {'address': '1123 Close Ave',
   'crossStreet': 'Watson',
   'lat': 40.82566103298058,
   'lng': -73.88261374008187,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.82566103298058,
     'lng': -73.88261374008187}],
   'distance': 3456,
   'postalCode': '10472',
   'cc': 'US',
   'city': 'Bronx',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['1123 Close Ave (Watson)',
    'Bronx, NY 10472',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d176941735',
    'name': 'Gym',
    'pluralName': 'Gyms',
    'shortName': 'Gym',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1595125479',
  'hasPerk': False},
 {'id': '4c7d06833e90a1cd540544be',
  'name': 'Alumni Gymnasium - CUNY Bronx Community College',
  'location': {'address': '2155 Dr Martin L King Jr Blvd',
   'cross

In [75]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
venues2 = results2['response']['venues']

# tranform venues into a dataframe
df5 = json_normalize(venues2)
df5.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,4cf19de26195721e9bc442c1,Daro's Extreme Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1595125479,False,1123 Close Ave,Watson,40.825661,-73.882614,"[{'label': 'display', 'lat': 40.82566103298058...",3456,10472,US,Bronx,NY,United States,"[1123 Close Ave (Watson), Bronx, NY 10472, Uni...",NaN,NaN
1,4c7d06833e90a1cd540544be,Alumni Gymnasium - CUNY Bronx Community College,"[{'id': '4bf58dd8d48988d1b2941735', 'name': 'C...",v-1595125479,False,2155 Dr Martin L King Jr Blvd,181 st,40.856683,-73.911255,"[{'label': 'display', 'lat': 40.85668339161032...",1026,10453,US,Bronx,NY,United States,"[2155 Dr Martin L King Jr Blvd (181 st), Bronx...",University Heights,NaN
2,518d102f498e55541f34e272,The Bayas Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1595125479,False,NaN,NaN,40.848153,-73.907015,"[{'label': 'display', 'lat': 40.848153, 'lng':...",893,NaN,US,Bronx,NY,United States,"[Bronx, NY, United States]",NaN,NaN
3,4afc1d83f964a520ef1f22e3,Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1595125479,False,1 Morningside Dr,Avalon Morningside Park,40.802107,-73.961297,"[{'label': 'display', 'lat': 40.80210712023747...",7761,10025,US,New York,NY,United States,"[1 Morningside Dr (Avalon Morningside Park), N...",NaN,NaN
4,5af0cd4b1ffe97002caf0eac,Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1595125479,False,NaN,NaN,40.713591,-74.044625,"[{'label': 'display', 'lat': 40.71359112321253...",19833,07302,US,Jersey City,NJ,United States,"[Jersey City, NJ 07302, United States]",NaN,NaN


In [76]:
df5_= df5[["name","location.address","location.lat","location.lng","location.formattedAddress"]]

In [77]:
df5_.rename(columns={"location.address": "address","location.lat":"Latitude","location.lng":"Longitude","location.formattedAddress":"full address"}, errors="raise", inplace=True)

### final data frame for gyms

In [78]:
df5_.head()

,name,address,Latitude,Longitude,full address
0,Daro's Extreme Gym,1123 Close Ave,40.825661,-73.882614,"[1123 Close Ave (Watson), Bronx, NY 10472, Uni..."
1,Alumni Gymnasium - CUNY Bronx Community College,2155 Dr Martin L King Jr Blvd,40.856683,-73.911255,"[2155 Dr Martin L King Jr Blvd (181 st), Bronx..."
2,The Bayas Gym,NaN,40.848153,-73.907015,"[Bronx, NY, United States]"
3,Gym,1 Morningside Dr,40.802107,-73.961297,"[1 Morningside Dr (Avalon Morningside Park), N..."
4,Gym,NaN,40.713591,-74.044625,"[Jersey City, NJ 07302, United States]"


In [79]:
df5_.shape

(50, 5)

In [80]:
df5_.dropna(inplace=True)

C:\Users\jake brown\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [81]:
df5_.shape

(41, 5)

### Map of NYC with the gyms we chose 

In [85]:
ny_map4 = folium.Map(location=[latitude, longitude], zoom_start=11.5)

# add markers to map
for name, latitude, longitude in zip(df5_['name'], df5_['Latitude'], df5_['Longitude']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=4,
        popup=label,
        color='purple',
        fill=True,
        fill_color='black',
        fill_opacity=0.7,
        parse_html=False).add_to(ny_map4)  
      
ny_map4

### Final Map with all the data such as: hotels, zoos, chinese restaurants and gyms

In [112]:
ny_map5 = folium.Map(location=[latitude, longitude], zoom_start=13)

for name, latitude, longitude in zip(df2['name'], df2['Latitude'], df2['Longitude']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color="black",
        fill_opacity=0.7,
        parse_html=False).add_to(ny_map5) 
    
    # add markers to map
for name, latitude, longitude in zip(df3_['name'], df3_['Latitude'], df3_['Longitude']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=4,
        popup=label,
        color='red',
        fill=True,
        fill_color="black",
        fill_opacity=0.7,
        parse_html=False).add_to(ny_map5)
    
for name, latitude, longitude in zip(df4_['name'], df4_['Latitude'], df4_['Longitude']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=4,
        popup=label,
        color='green',
        fill=True,
        fill_color="black",
        fill_opacity=0.7,
        parse_html=False).add_to(ny_map5)  

# add markers to map
for name, latitude, longitude in zip(df5_['name'], df5_['Latitude'], df5_['Longitude']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=4,
        popup=label,
        color='purple',
        fill=True,
        fill_color="black",
        fill_opacity=0.7,
        parse_html=False).add_to(ny_map5)  
      
ny_map5

### CONCLUSION AND RECOMMENDATIONS

For our study we have a lot of places next to each other’s and for our recommendations we will recommend our clients venues like: the plaza hotel, Sanctuary hotel, the Kimberly hotel, gym central park place, AKA gym, the Gym at the Atlas, central park zoo - tropic zoo, central park zoo - Tish children’s zoo, central park zoo – snow leopard, Nong Xin Chinese Kitchen, Chinese Take Out.
